# Finding Similar Items

## 3.1 Applications of Near-Neighbor Search

## 3.1.1 Jaccard Similarity of Sets

$$Jaccard\ Similarity = \frac{|A\cap B|}{|A\cup B|}$$

In [1]:
def jaccard_sim(a, b):
    
    return len(a.intersection(b)) / len(a.union(b))

In [2]:
a = set([1, 2, 3, 4, 5])
b = set([3, 4, 7, 8])

a_intersection_b = a.intersection(b)
a_union_b = a.union(b)

print("Intersection: {0}\nUnion: {1}\nJaccard Similarity = {2}".format(a_intersection_b, 
                                                                       a_union_b,
                                                                       jaccard_sim(a, b)))

Intersection: {3, 4}
Union: {1, 2, 3, 4, 5, 7, 8}
Jaccard Similarity = 0.2857142857142857


### 3.1.2 Similarity of Documents

### 3.1.3 Collaborative Filtering as a Similar-Sets Problem




**ver introduction to recommender systems**

In [3]:
def jaccard_bag_sim(a, b):
    
    intersection_sum = sum((a & b).values())
    union_sum = sum(a.values()) + sum(b.values())
    
    return intersection_sum / union_sum

In [4]:
from collections import Counter

a = Counter('aaab')
b = Counter('aabbc')

In [5]:
jaccard_bag_sim(a, b)

0.3333333333333333

## 3.2 Shingling of Documents

### 3.2.1 k-Shingles

In [101]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import codecs
import re
import nltk.data

In [18]:
with codecs.open('dom_casmurro.txt', 'r', 'utf-8') as f:
    
    dom_casmurro = f.readlines()
    
with codecs.open('perto_do_coracao_selvagem.txt', 'r', 'utf-8') as f:
    
    perto_coracao = f.readlines()

In [38]:
dom_casmurro[:10]

['CAPÍTULO PRIMEIRO\n',
 'DO TÍTULO\n',
 ' \n',
 'Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no trem da Central um rapaz aqui do bairro, que eu conheço de vista e de chapéu. Cumprimentou-me, sentou-se ao pé de mim, falou da Lua e dos ministros, e acabou recitando-me versos. A viagem era curta, e os versos pode ser que não fossem inteiramente maus. Sucedeu, porém, que, como eu estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso.\n',
 ' \n',
 '— Continue, disse eu acordando.\n',
 ' \n',
 '— Já acabei, murmurou ele.\n',
 ' \n',
 '— São muito bonitos.\n']

In [61]:
re_empty_line = re.compile(r'^\s*$')
re_remove_chars = re.compile("[\r\n•]+")
re_chapter_title = re.compile("^[A-ZÁÉÍÓÚÃÕ\. ]+\s*$")

def clean_line(line):
    
    return re_remove_chars.sub("", line)

def is_good_line_casmurro(line):
    
    return re_empty_line.match(line) is None and\
           re_chapter_title.match(line) is None
    
def is_good_line_clarice(line):
    
    return re_empty_line.match(line) is None
    
def clean_and_filter_casmurro(lines):
    
    return [line for line in [clean_line(line) for line in lines] if \
            is_good_line_casmurro(line)]

def clean_and_filter_clarice(lines):
    
    return [line for line in [clean_line(line) for line in lines] if \
            is_good_line_clarice(line)]

In [62]:
clean_and_filter_casmurro(dom_casmurro[:10])

['Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no trem da Central um rapaz aqui do bairro, que eu conheço de vista e de chapéu. Cumprimentou-me, sentou-se ao pé de mim, falou da Lua e dos ministros, e acabou recitando-me versos. A viagem era curta, e os versos pode ser que não fossem inteiramente maus. Sucedeu, porém, que, como eu estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso.',
 '— Continue, disse eu acordando.',
 '— Já acabei, murmurou ele.',
 '— São muito bonitos.']

In [63]:
perto_coracao[:10]

['O PAI \n',
 '\n',
 '\n',
 '• • • \n',
 '\n',
 '\n',
 'A MAQUINA DO PAPAI batia tac-tac... tac-tac-tac... O relógio acordou emtin-dlen \n',
 'sem poeira. O silêncio arrastou-se zzzzzz. O guarda-roupa dizia o quê? roupa-roupa-roupa. \n',
 'Não, não. Entre o relógio, a máquina e o silêncio havia uma orelha à escuta, grande, cor-de- \n',
 'rosa e morta. Os três sons estavam ligados pela luz do dia e pelo ranger das folhinhas da \n']

In [64]:
clean_and_filter_clarice(perto_coracao[:10])

['O PAI ',
 'A MAQUINA DO PAPAI batia tac-tac... tac-tac-tac... O relógio acordou emtin-dlen ',
 'sem poeira. O silêncio arrastou-se zzzzzz. O guarda-roupa dizia o quê? roupa-roupa-roupa. ',
 'Não, não. Entre o relógio, a máquina e o silêncio havia uma orelha à escuta, grande, cor-de- ',
 'rosa e morta. Os três sons estavam ligados pela luz do dia e pelo ranger das folhinhas da ']

In [76]:
dom_casmurro = clean_and_filter_casmurro(dom_casmurro)
perto_coracao = clean_and_filter(perto_coracao)

In [104]:
sent_tokenizer = nltk.data.load("tokenizers/punkt/portuguese.pickle")

In [109]:
dom_casmurro = sent_tokenizer.tokenize(' '.join(dom_casmurro))
perto_coracao = sent_tokenizer.tokenize(' '.join(perto_coracao))

In [111]:
dom_casmurro[:5]

['Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no trem da Central um rapaz aqui do bairro, que eu conheço de vista e de chapéu.',
 'Cumprimentou-me, sentou-se ao pé de mim, falou da Lua e dos ministros, e acabou recitando-me versos.',
 'A viagem era curta, e os versos pode ser que não fossem inteiramente maus.',
 'Sucedeu, porém, que, como eu estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso.',
 '— Continue, disse eu acordando.']

In [112]:
cv = CountVectorizer(analyzer='word', # n-gram de words
                     ngram_range=(2, 2), # 2-gram
                     token_pattern='\w+') # considerar 'word' um ou mais caracteres

cv.fit(dom_casmurro + perto_coracao)

print("Tamanho do vocabulário: {}".format(len(cv.vocabulary_)))

Tamanho do vocabulário: 62931


In [113]:
line = dom_casmurro[33:34]
cv_bigrams = cv.transform(line)
df_cv_bigrams = pd.DataFrame(cv_bigrams.todense(), columns=cv.get_feature_names(), index=['count']).T

print("Linha:\n{}".format(line))

Linha:
['Uso louça velha e mobília velha.']


In [114]:
df_cv_bigrams[df_cv_bigrams['count'] > 0]

,count
e mobília,1
louça velha,1
mobília velha,1
uso louça,1
velha e,1


## 3.2.3 Hashing Shingles

In [11]:
from sklearn.feature_extraction.text import HashingVectorizer

In [12]:
hv = HashingVectorizer(analyzer='word', # n-gram de words
                     ngram_range=(2, 2), # 2-gram
                     token_pattern='\w+')

hv.fit([dom_casmurro, perto_coracao])

HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_range=(2, 2), non_negative=False,
         norm='l2', preprocessor=None, stop_words=None, strip_accents=None,
         token_pattern='\\w+', tokenizer=None)

In [13]:
hv_bigrams = hv.transform([dom_casmurro])

In [14]:
import sys

print("Perto do Coração Selvagem vocabulary size: {0} megabytes".format(sys.getsizeof(perto_coracao) / 10**6))
print("Dom Casmurro size: {0} megabytes".format(sys.getsizeof(dom_casmurro) / 10**6))
print("CountVectorizer vocabulary size: {0} megabytes".format(sys.getsizeof(cv.vocabulary_) / 10**6))

Perto do Coração Selvagem vocabulary size: 0.582302 megabytes
Dom Casmurro size: 0.750664 megabytes
CountVectorizer vocabulary size: 2.621544 megabytes


## 3.3 Similarity-Preserving Summaries of Sets

### 3.3.1 Matrix Representation of Sets

In [15]:
from sklearn.feature_extraction import DictVectorizer

In [16]:
dv = DictVectorizer(sparse=False)

D = [{'a':1, 'd':1}, {'c': 1}, {'b': 1, 'd': 1, 'e': 1}, {'a': 1, 'c': 1, 'd': 1}]

X = dv.fit_transform(D)
X # diferente da notação no livro, os conjuntos ficam representados por linhas, e não colunas

array([[ 1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  1.],
       [ 1.,  0.,  1.,  1.,  0.]])

In [17]:
dv.inverse_transform(X)

[{'a': 1.0, 'd': 1.0},
 {'c': 1.0},
 {'b': 1.0, 'd': 1.0, 'e': 1.0},
 {'a': 1.0, 'c': 1.0, 'd': 1.0}]

### 3.3.2 Minhashing

Ver: http://mccormickml.com/2015/06/12/minhash-tutorial-with-python-code/

### 3.3.3 Minhashing and Jaccard Similarity

### 3.3.4 Minhash Signatures

### 3.3.5 Computing Minhash Signatures

## 3.4 Locality-Sensitive Hashing for Documents